In [1]:
1+1

2

In [2]:
import os
import logging
from dotenv import load_dotenv
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, HumanMessage, ToolMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain.agents import Tool
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, END


# 환경 변수
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# logging.basicConfig(level=logging.DEBUG) # 필요시 주석 해제
logging.getLogger("langchain").setLevel(logging.DEBUG)


# 도구 함수 정의
def add_two(x: str) -> str:
    try:
        return str(float(x) + 2)
    except Exception:
        return "도구 실행 실패: 숫자만 입력해 주세요."


def population_lookup(city: str) -> str:
    data = {"서울": "9,515,000명 (2023년 기준)", "부산": "3,343,000명 (2023년 기준)"}
    # `raise ValueError` 대신 특정 메시지를 반환하여 에이전트가 이를 처리하도록 유도
    if city.strip().lower() in data:
        return data.get(city.strip().lower())
    else:
        # 에이전트가 이 메시지를 보고 LLM fallback을 고려하도록 유도
        return f"{city}의 인구 정보를 찾을 수 없습니다."


# 메시지 상태 정의
class AgentState(TypedDict):
    messages: List[BaseMessage]
    # 추가: 에이전트가 도구 사용에 실패했는지 여부를 나타내는 플래그
    tool_failure: bool


# 도구 리스트 정의
tools = [
    Tool(name="AddTwo", func=add_two, description="숫자 입력 시 2를 더합니다."),
    Tool(
        name="PopulationLookup",
        func=population_lookup,
        description="도시 인구를 조회합니다.",
    ),
]


# LLM 정의 (ReAct agent는 verbose=True만 있어도 추론 흐름 출력)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, verbose=True)


# Fallback용 순수 LLM 체인 생성
def llm_only_response(state: AgentState):
    print("--- LLM Fallback 실행 ---")
    # 마지막 사용자 메시지 가져오기
    last_message = state["messages"][-1]
    if isinstance(last_message, HumanMessage):
        user_query = last_message.content
    elif isinstance(last_message, ToolMessage):
        # 도구 실행 실패로 넘어온 경우, 원래의 사용자 쿼리를 찾아서 LLM에 전달
        for msg in reversed(state["messages"]):
            if isinstance(msg, HumanMessage):
                user_query = msg.content
                break
        else:  # HumanMessage를 찾지 못한 경우
            user_query = "사용자의 마지막 질문을 알 수 없습니다."

    else:  # 기타 메시지 타입 (예: AIMessage가 실패한 경우 등)
        user_query = str(last_message)

    # LLM에게 직접 질문
    response = llm.invoke(
        f"다음 질문에 답해주세요: {user_query}. 가능한 경우, 도구 사용에 실패했음을 언급하고, 직접 답해주세요."
    )

    return {"messages": state["messages"] + [response]}


# ReAct Agent 생성
react_agent = create_react_agent(model=llm, tools=tools, debug=True)


# 에이전트 노드에서 다음 상태를 결정하는 함수
def route_agent(state: AgentState):
    last_message = state["messages"][-1]

    # 마지막 메시지가 ToolMessage이고 오류가 포함되어 있거나,
    # AIMessage가 도구 사용을 계획했으나 오류가 발생했다고 판단되면 fallback
    if isinstance(last_message, ToolMessage) and (
        "찾을 수 없습니다" in last_message.content or "실패" in last_message.content
    ):
        return "llm_fallback"

    # 일반적인 에이전트의 최종 응답 (AIMessage)
    if isinstance(last_message, AIMessage):
        return "end"

    # 에이전트가 추가 도구 사용이나 생각을 계속해야 하는 경우
    return "agent"


# LangGraph 구성
graph = StateGraph(AgentState)

graph.add_node("agent", react_agent)
graph.add_node("llm_fallback", llm_only_response)


graph.set_entry_point("agent")

# 조건부 에지 추가
graph.add_conditional_edges(
    "agent",  # "agent" 노드 이후에
    route_agent,  # route_agent 함수를 통해 다음 노드를 결정
    {
        "llm_fallback": "llm_fallback",  # route_agent가 "llm_fallback"을 반환하면 llm_fallback 노드로
        "end": END,  # route_agent가 "end"를 반환하면 END로
        "agent": "agent",  # route_agent가 "agent"를 반환하면 다시 agent 노드로 (재시도 또는 추가 도구 사용)
    },
)

# llm_fallback 노드에서 작업 완료 후 END로 이동
graph.add_edge("llm_fallback", END)


# 그래프 컴파일
app = graph.compile()


# 테스트 실행
max_iterations = 3
recursion_limit = 2 * max_iterations + 1

print("\n--- 테스트 1: 성공적인 도구 사용 (부산) ---")
try:
    response1 = app.invoke(
        {"messages": [HumanMessage(content="부산 인구는?")]},
        {"recursion_limit": recursion_limit},
    )
    for m in response1["messages"]:
        print(m.__class__.__name__, ":", getattr(m, "content", repr(m)))
except Exception as e:
    print(f"Agent 실행 중 오류 발생: {e}")
print("---")

print("\n--- 테스트 2: 도구 실패 및 Fallback (대구) ---")
try:
    response2 = app.invoke(
        {"messages": [HumanMessage(content="대구 인구는?")]},
        {"recursion_limit": recursion_limit},
    )
    for m in response2["messages"]:
        print(m.__class__.__name__, ":", getattr(m, "content", repr(m)))
except Exception as e:
    print(f"Agent 실행 중 오류 발생: {e}")
print("---")

print("\n--- 테스트 3: 도구 성공 (숫자에 2 더하기) ---")
try:
    response3 = app.invoke(
        {"messages": [HumanMessage(content="숫자 5에 2를 더해줘.")]},
        {"recursion_limit": recursion_limit},
    )
    for m in response3["messages"]:
        print(m.__class__.__name__, ":", getattr(m, "content", repr(m)))
except Exception as e:
    print(f"Agent 실행 중 오류 발생: {e}")
print("---")

print("\n--- 테스트 4: 도구 실패 (문자에 2 더하기) 및 Fallback ---")
try:
    response4 = app.invoke(
        {"messages": [HumanMessage(content="문자 'abc'에 2를 더해줘.")]},
        {"recursion_limit": recursion_limit},
    )
    for m in response4["messages"]:
        print(m.__class__.__name__, ":", getattr(m, "content", repr(m)))
except Exception as e:
    print(f"Agent 실행 중 오류 발생: {e}")
print("---")

print("\n--- 테스트 5: 일반적인 질문 (도구 필요 없음) ---")
try:
    response5 = app.invoke(
        {"messages": [HumanMessage(content="날씨가 어때?")]},
        {"recursion_limit": recursion_limit},
    )
    for m in response5["messages"]:
        print(m.__class__.__name__, ":", getattr(m, "content", repr(m)))
except Exception as e:
    print(f"Agent 실행 중 오류 발생: {e}")
print("---")


--- 테스트 1: 성공적인 도구 사용 (부산) ---
[values] {'messages': [HumanMessage(content='부산 인구는?', additional_kwargs={}, response_metadata={}, id='450cfc93-b092-45dd-86e7-bf07c35ea3bd')]}
[updates] {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_IcaMBHHr8tbyJaPeaLtdzThs', 'function': {'arguments': '{"__arg1":"부산"}', 'name': 'PopulationLookup'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 88, 'total_tokens': 106, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BpYrZAflR3feg8prYLsvt358o7WNP', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9eb29b8d-3ef9-4dae-88ea-107a66877701-0', tool_calls=[{'name